# Guide : Mise en place de la documentation Sphinx + CI/CD pour le projet Mangetamain

Ce notebook contient toutes les étapes (0 à 14) pour mettre en place la génération et la publication de la documentation du projet Mangetamain.

## 0) Où travailler
- Ouvre un terminal **à la racine du repo git** (ex. `~/projects/mangetamain`).

## 1) Ajouter les dépendances docs dans `pyproject.toml`
**Où** : fichier `pyproject.toml` (racine)

```toml
[tool.uv.dependencies.docs]
sphinx = ">=7.0.0"
sphinx-rtd-theme = "*"
sphinx-autodoc-typehints = "*"
sphinxcontrib-napoleon = "*"
```
(Si tu utilises `poetry`, mets-les dans `[tool.poetry.group.docs.dependencies]`).

## 2) Installer `uv` et les dépendances docs
**Où** : terminal

```bash
pip install --user uv       # si uv pas déjà installé
uv sync --group docs
```

## 3) Initialiser Sphinx
**Où** : terminal (racine)

```bash
uv run sphinx-quickstart -q -p "Mangetamain" -a "Equipe Mangetamain" -v "0.1.0" docs
```

Cela crée le dossier `docs/` avec `conf.py`, `index.rst`, etc.

## 4) Configurer `docs/conf.py`
**Où** : éditeur → `docs/conf.py`

```python
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

project = "Mangetamain"
author = "Equipe Mangetamain"
release = "0.1.0"

extensions = [
    "sphinx.ext.autodoc",
    "sphinx.ext.napoleon",
    "sphinx.ext.viewcode",
    "sphinx_autodoc_typehints",
    "sphinx.ext.autosummary",
]

autosummary_generate = True
html_theme = "sphinx_rtd_theme"
```

## 5) Rédiger les docstrings dans ton code
**Où** : fichiers Python sous `src/`

Exemple :

```python
def score_recipe(ingredients: list[str], time_min: int) -> float:
    """
    Calcule un score de recette.

    Args:
        ingredients (list[str]): liste des ingrédients.
        time_min (int): temps de préparation en minutes.

    Returns:
        float: score estimé entre 0 et 1.
    """
```

## 6) Générer automatiquement les fichiers `.rst`
**Où** : terminal

```bash
uv run sphinx-apidoc -o docs/ src/ --separate --module-first
```

## 7) Construire la doc HTML localement
**Où** : terminal

```bash
uv run sphinx-build -b html docs/ docs/_build/html
```
Puis ouvre `docs/_build/html/index.html` dans un navigateur.

Optionnel :

```bash
uv run pip install sphinx-autobuild
uv run sphinx-autobuild docs/ docs/_build/html
```

## 8) Ajouter un job "docs build" dans la CI
**Où** : fichier `.github/workflows/docs.yml`

```yaml
name: Docs

on:
  push:
    branches: ["main"]
  pull_request:
    branches: ["main"]

jobs:
  build-docs:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with:
          python-version: "3.11"
      - run: pip install uv
      - run: uv sync --group docs
      - run: uv run sphinx-build -b html docs/ docs/_build/html
```

## 9) Déployer la doc sur GitHub Pages
**Où** : fichier `.github/workflows/publish-docs.yml`

```yaml
name: Publish docs

on:
  push:
    branches: ["main"]

jobs:
  publish:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
        with:
          persist-credentials: false
      - uses: actions/setup-python@v5
        with:
          python-version: "3.11"
      - run: pip install uv
      - run: uv sync --group docs
      - run: uv run sphinx-build -b html docs/ docs/_build/html
      - name: Deploy to GitHub Pages
        uses: peaceiris/actions-gh-pages@v4
        with:
          github_token: ${{ secrets.GITHUB_TOKEN }}
          publish_dir: ./docs/_build/html
```

## 10) Intégrer la vérification docs dans la CI principale
Ajoute un step `uv run sphinx-build` à la fin du job `lint-test` dans `.github/workflows/ci.yml`.

## 11) CI pour lint (ruff) et tests (pytest)
**Où** : fichier `.github/workflows/ci.yml`

```yaml
name: CI

on:
  push:
    branches: ["main"]
  pull_request:
    branches: ["main"]

jobs:
  lint-test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with:
          python-version: "3.11"
      - run: pip install uv
      - run: uv sync --group dev
      - run: uv run ruff check .
      - run: uv run pytest --cov=src --cov-report=xml --cov-report=term --cov-fail-under=90
```

## 12) Commit et push
```bash
git add .
git commit -m "Add Sphinx docs and CI workflows"
git push origin main
```

## 13) Bonnes pratiques
- Ajouter un lien vers la doc dans le `README.md`
- Écrire des pages `.rst` (guides, how-to) dans `docs/`
- Utiliser `autodoc_mock_imports` si certaines libs sont trop lourdes
- Ajouter des badges CI et Pages dans le README

## 14) Checklist
- [ ] `pyproject.toml` mis à jour avec dépendances docs/dev  
- [ ] `docs/` créé, `conf.py` configuré  
- [ ] Docstrings écrits dans `src/`  
- [ ] Génération locale OK (`sphinx-build`)  
- [ ] CI docs OK (job passe)  
- [ ] Pages déployées sur GitHub Pages  
- [ ] README contient le lien vers la doc  